In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem.AllChem import EmbedMolecule, MMFFOptimizeMolecule
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = True

from molclub.conf_tools import etkdg
from molclub.compute import orca_dft

ORCA_DIR = '/Users/ozone/bin/_orca_5_0_3'

## utils

In [45]:
def get_substructure(mol, atom_idx, rank):
    if Chem.MolToSmiles(mol) != Chem.MolToSmiles(Chem.AddHs(mol)):
        raise ValueError('mol must contain Hs')
    atom = mol.GetAtomWithIdx(atom_idx)
    idxs = [atom_idx]
    neighbors = atom.GetNeighbors()
    # collect the atom indices of the atoms that will be part of the substructure
    for _ in range(rank):
        idxs = list(set(idxs + [neighbor.GetIdx() for neighbor in neighbors]))
        new_neighbors = []
        for neighbor in neighbors:
            new_neighbors += neighbor.GetNeighbors()
        new_neighbors = set([neighbor.GetIdx() for neighbor in new_neighbors])
        new_neighbors = [idx for idx in new_neighbors if idx not in idxs]
        neighbors = [mol.GetAtomWithIdx(idx) for idx in new_neighbors]

    # create a copy of the mol only containing relevant atoms
    rw_mol = Chem.RWMol(mol)
    for i in reversed(range(rw_mol.GetNumAtoms())):
        if i not in idxs:
            rw_mol.RemoveAtom(i)
    
    return rw_mol.GetMol()

def get_exact_atom_id(mol, atom_idx, df_atoms):
    atom_type = [Chem.MolToSmarts(get_substructure(mol, atom_idx, rank)) for rank in range(4)]
    matches = df_atoms.loc[df_atoms[['r0_submol', 'r1_submol', 'r2_submol', 'r3_submol']].apply(lambda x: x.to_list() == atom_type, axis=1)]
    match = matches.iloc[0]

    return match.atom_id

def get_bond_length(mol, atom_idx_1, atom_idx_2):
    conf = mol.GetConformer()
    positions = conf.GetPositions()
    atom_pos_1 = positions[atom_idx_1]
    atom_pos_2 = positions[atom_idx_2]
    bond_length = np.linalg.norm(atom_pos_2 - atom_pos_1)

    return bond_length

## X-H bonds

working off of SMILES and regenerating coordinates each time is bad
1. a lot of repeated work
2. hard to keep data around to update df_atoms as atom_ids will change

instead save mols to sdf and keep a repository of molecules
1. work is only done once
2. easier to rebuild df_atoms

In [87]:
SINGLE_ELEMENTS = [
    'B',
    'C',
    'N',
    'O',
    'F',
    '[SiH4]',
    'P',
    'S',
    'Cl',
    '[SeH2]',
    'Br',
    'I'
]

SINGLE_ELEMENTS = ['[nH]1cncc1']

df_atoms = set()
for element in SINGLE_ELEMENTS:
    mol = Chem.MolFromSmiles(element)
    mol = Chem.AddHs(mol)
    for idx in range(mol.GetNumAtoms()):
        atom_type = tuple([Chem.MolToSmarts(get_substructure(mol, idx, rank)) for rank in range(4)])
        df_atoms.add(atom_type)

In [88]:
df_atoms = pd.DataFrame(df_atoms, columns=['r0_submol', 'r1_submol', 'r2_submol', 'r3_submol'])
df_atoms['symbol'] = df_atoms.r0_submol.apply(lambda x: Chem.MolFromSmarts(x).GetAtomWithIdx(0).GetSymbol())
symbols = df_atoms.symbol.to_list()
unique_symbols = set(symbols)
symbol_idx = {symbol: symbols.index(symbol) for symbol in unique_symbols}
df_atoms = df_atoms.sort_values(['symbol', 'r0_submol', 'r1_submol', 'r2_submol', 'r3_submol']).reset_index(drop=True)
df_atoms['atom_id'] = df_atoms.apply(lambda x: f'{x.symbol}{x.name - symbol_idx[x.symbol] + 1}', axis=1)

df_atoms

,r0_submol,r1_submol,r2_submol,r3_submol,symbol,atom_id
0,[#6],[#7]:[#6](:[#6])-[H],[#7]1(:[#6]:[#7]:[#6](:[#6]:1-[H])-[H])-[H],[#7]1(:[#6](:[#7]:[#6](:[#6]:1-[H])-[H])-[H])-[H],C,C0
1,[#6],[#7]:[#6](:[#6])-[H],[#7]1:[#6]:[#7]:[#6](:[#6]:1-[H])-[H],[#7]1(:[#6](:[#7]:[#6](:[#6]:1-[H])-[H])-[H])-[H],C,C1
2,[#6],[#7]:[#6](:[#7])-[H],[#7]1(:[#6](:[#7]:[#6]:[#6]:1)-[H])-[H],[#7]1(:[#6](:[#7]:[#6](:[#6]:1-[H])-[H])-[H])-[H],C,C2
3,[H],[#6]-[H],[#7]:[#6](:[#6])-[H],[#7]1(:[#6]:[#7]:[#6](:[#6]:1-[H])-[H])-[H],H,H1
4,[H],[#6]-[H],[#7]:[#6](:[#6])-[H],[#7]1:[#6]:[#7]:[#6](:[#6]:1-[H])-[H],H,H2
5,[H],[#6]-[H],[#7]:[#6](:[#7])-[H],[#7]1(:[#6](:[#7]:[#6]:[#6]:1)-[H])-[H],H,H3
6,[H],[#7]-[H],[#7](:[#6])(:[#6])-[H],[#7]1(:[#6](:[#7]:[#6]:[#6]:1-[H])-[H])-[H],H,H4
7,[#7],[#6]:[#7]:[#6],[#7]1:[#6](:[#7]:[#6](:[#6]:1)-[H])-[H],[#7]1(:[#6](:[#7]:[#6](:[#6]:1-[H])-[H])-[H])-[H],N,N8
8,[#7],[#7](:[#6])(:[#6])-[H],[#7]1(:[#6](:[#7]:[#6]:[#6]:1-[H])-[H])-[H],[#7]1(:[#6](:[#7]:[#6](:[#6]:1-[H])-[H])-[H])-[H],N,N9


In [89]:
df_bonds = []
for element in SINGLE_ELEMENTS:
    mol = Chem.MolFromSmiles(element)
    mol = Chem.AddHs(mol)
    mol = etkdg.generate_conformers(mol, 1)[0]
    mol, _ = orca_dft.opt(mol, ORCA_DIR)

    for idx in range(mol.GetNumAtoms()):
        for jdx in range(mol.GetNumAtoms()):
            if jdx > idx and mol.GetBondBetweenAtoms(idx, jdx) is not None:
                atom_id_i = get_exact_atom_id(mol, idx, df_atoms)
                atom_id_j = get_exact_atom_id(mol, jdx, df_atoms)
                sorted_atom_ids = sorted([atom_id_i, atom_id_j])
                bond_length = get_bond_length(mol, idx, jdx)
                df_bonds.append(sorted_atom_ids + [bond_length])

In [90]:
df_bonds = pd.DataFrame(df_bonds, columns=['atom_id_1', 'atom_id_2', 'bond_length'])
df_bonds = df_bonds.groupby(['atom_id_1', 'atom_id_2']).agg({'bond_length': list}).reset_index()
df_bonds['max-min'] = df_bonds.bond_length.apply(lambda x: max(x) - min(x))
df_bonds.bond_length = df_bonds.bond_length.apply(np.mean)

df_bonds

,atom_id_1,atom_id_2,bond_length,max-min
0,C0,C1,1.371022,0.0
1,C0,H1,1.080121,0.0
2,C0,N9,1.375187,0.0
3,C1,H2,1.082100,0.0
4,C1,N8,1.382135,0.0
5,C2,H3,1.083049,0.0
6,C2,N8,1.322687,0.0
7,C2,N9,1.359083,0.0
8,H4,N9,1.009978,0.0


In [ ]:
# determine best matching existing atom type by finding ratio of len(mcs)/(len(mol_query) + len(mol_template))*2